## Imports

In [31]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pickle
import pandas as pd
import numpy as np
from urllib.request import urlopen
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

## Start driver

In [32]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver   
            
driver = webdriver.Chrome(chromedriver)

open_table_url = "https://www.opentable.com/chicago-illinois-restaurant-listings"

driver.get(open_table_url)

## Retrieve list of links from each page

This code pulls each link from a page and then clicks to the next page to retrieve the links until it can no longer click next.

In [1]:
list_of_chicago_links = []

while True:
    try:
        list_of_links = driver.find_elements_by_xpath("(//span[@class='rest-row-name-text'])")

        for link in list_of_links:
            list_of_chicago_links.append(link.text)

        driver.find_elements_by_css_selector("a.pagination-link.pagination-next")[0].click()
        time.sleep(5)
    
    except TimeoutException:
        print("Continuing...")

Write links into pickle file

In [34]:
pickle.dump(list_of_chicago_links, open("list_of_chicago_links2.p", "wb"))

In [ ]:
def open_table_page_to_links():
    list_of_chicago_links = []
    list_of_links = driver.find_elements_by_xpath("(//span[@class='rest-row-name-text'])")

    for link in list_of_links:
    list_of_chicago_links.append(link.text)

In [ ]:
links = open_table_page_to_links()
all_restaurant_info = []
for link in links:
    restaurant_info = get_restaurant_info(link)
    all_restaurant_info.append(restaurant_info)

Reads links pickle file

In [3]:
list_of_linktext = pickle.load(open('list_of_chicago_links2.p', 'rb'))

## Functions

In [6]:
def make_soup ():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

def switch_to_window(index):
    driver.switch_to.window(driver.window_handles[index])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup
    
def switch_to_original_window():
    time.sleep(5)
    driver.back()
    driver.switch_to.window(driver.window_handles[0])
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(8)
    return soup
    
def click_hours():
    hours_button = driver.find_element_by_link_text('View Hours, Transportation, and Other Details')
    hours_button.click()
    time.sleep(1)
    return hours_button

def open_table_element(soup, index):
    output_list = []
    element = soup.find_all("span", {"class":"font-weight-medium"})[index].next_sibling.next_sibling.text
    output_list.append(element)
    return output_list

def open_table_rating(soup, index):
    output_list = []
    try:
        element = soup.find_all("h6", {"class":"color-light"})[index].next_sibling.text
        element = float(element)
        output_list.append(element)

    except IndexError:
        print("Element not on page...continuing script")
        output_list.append(np.nan) 
    return output_list

def get_reviews():
    try:

        overall = []

        for element in soup.find_all(class_= 'reviews-overall-score'):
            rating = element.text
            rating = float(rating)
            overall.append(rating)

        overall_rating.append(overall)

    except NoSuchElementException:
        print("Element not on page..continuing script")
        overall_rating.append(np.nan)
        
def get_prices():
    try:
        menu_button = driver.find_element_by_id('rest-menu-view-full')
        menu_button.click()
        time.sleep(5)
        
        prices = []

        for element in soup.find_all(class_= 'rest-menu-item-price'):
            price = element.text
            price = price.replace('$','')
            price = float(price)
            prices.append(price)
            
        if len(prices) > 0:
            price_avg = sum(prices)/(len(prices))
            price_avg_list.append(price_avg)
        
        else:
            price_avg_list.append(np.nan)

        price_avg = sum(prices)/(len(prices))
        price_avg_list.append(price_avg)
        
    except NoSuchElementException:
        print("Element not on page..continuing script")
        price_avg_list.append(np.nan)
        
    return price_avg_list

def open_table_rating(soup, index):
    output_list = []
    try:
        element = soup_find_all_element("h6", {"class":"color-light"})[index].next_sibling.text
        element = float(element)
        output_list.append(element)

    except IndexError:
        print("Element not on page...continuing script")
        output_list.append(np.nan) 
    return output_list

def open_table_element(soup, index):
    output_list = []
    element = soup_find_all_element('span', 'class', 'font-weight-medium')[index].next_sibling.next_sibling.text
    output_list.append(element)
    return output_list

## Make soup

In [ ]:
make_soup()

## Read in more pickle files

In [14]:
value_list = pickle.load(open("value_list.p", "rb"))
price_avg_list = pickle.load(open("price_avg_list.p", "rb" ))
dining_style_list = pickle.load(open( "dining_style_list.p", "rb" ))
dress_code_list = pickle.load(open( "dress_code_list.p", "rb" ))
overall_rating = pickle.load(open( "overall_rating.p", "rb" ))
food_list = pickle.load(open( "food_list.p", "rb" ))
service_list = pickle.load(open( "service_list.p", "rb" ))
ambience_list = pickle.load(open( "ambience_list.p", "rb" ))
noise_list = pickle.load(open( "noise_list.p", "rb" ))
cuisine_list = pickle.load(open( "cuisine_list.p", "rb" ))

## Scrape data

In [7]:
"""The below commented areas weren't commented out. I commented them out so I could keep appending data to them and not overwrite anything"""

# price_avg_list = []
# dining_style_list = []
# cuisine_list = []
# dress_code_list = []
# overall_rating = []
# food_list = []
# service_list = []
# ambience_list = []
# value_list = []
# noise_list = []

# counter = 1

# link_text_list = list_of_linktext[74:76]
# link_text_list2 = list_of_linktext[76:100]
# link_text_list4 = list_of_linktext[135:200]
# link_text_list5 = list_of_linktext[198:205]
link_text_list8 = list_of_linktext[205:]

for linktext in link_text_list8:
    while True:
        
        try:

            driver.find_element_by_link_text(linktext).click()
            time.sleep(5)

            switch_to_window(-1)
            
            make_soup()
            
            get_prices()

            get_reviews()

            #food rating
            open_table_rating(soup, 0)
            
            #service rating
            open_table_rating(soup, 1)
            
            #ambience rating
            open_table_rating(soup, 2)

            #value rating
            open_table_rating(soup, 3)
            
            #noise rating
            open_table_rating(soup,4)

            hours_button = driver.find_element_by_link_text('View Hours, Transportation, and Other Details')
            hours_button.click()
            time.sleep(8)

            dining_style_list = open_table_element(soup, 0)
            
            cuisine_list = open_table_element(soup, 1)
            
            dress_code_list = open_table_element(soup, 4)

            switch_to_original_window()

            break

        except (NoSuchElementException, TimeoutException):

            driver.switch_to.window(driver.window_handles[0])
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            driver.execute_script('window.scrollTo(0, 0)')
            driver.find_elements_by_css_selector("a.pagination-link.pagination-next")[0].click()
            time.sleep(8)
    
    print('Status: Done.')
    print(linktext)
    counter+=1

## Pickle scraped data

In [55]:
import pickle

pickle.dump(value_list, open( "value_list.p", "wb" ))
pickle.dump(price_avg_list, open( "price_avg_list.p", "wb" ))
pickle.dump(dining_style_list, open( "dining_style_list.p", "wb" ))
pickle.dump(dress_code_list, open( "dress_code_list.p", "wb" ))
pickle.dump(overall_rating, open( "overall_rating.p", "wb" ))
pickle.dump(food_list, open( "food_list.p", "wb" ))
pickle.dump(service_list, open( "service_list.p", "wb" ))
pickle.dump(ambience_list, open( "ambience_list.p", "wb" ))
pickle.dump(noise_list, open( "noise_list.p", "wb" ))
pickle.dump(cuisine_list, open( "cuisine_list.p", "wb" ))

## Close driver

In [5]:
driver.close